# ASV Overlap Venn Diagram

In [10]:
# Import Python packages
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import biom
from biom import load_table
import numpy as np
from scipy.stats import pearsonr
from matplotlib_venn import venn2
import matplotlib.patches as mpatches
from scipy.stats import mannwhitneyu
from biom import Table
from gemelli.rpca import rpca
from scipy.spatial.distance import euclidean
import re
import matplotlib.colors as mcolors
from scipy.stats import fisher_exact


In [11]:
# Load the metadata
metadata_path = '../Data/Metadata/updated_clean_ant_skin_metadata_microbiome_type.tab'
metadata = pd.read_csv(metadata_path, sep='\t')

metadata['#sample-id'] = metadata['#sample-id'].str.replace('_', '')
# Set Sample-ID as the index for the metadata dataframe 
metadata = metadata.set_index('#sample-id')

# Create group column based on case_type to simplify group names
metadata['group'] = metadata['case_type'].map({
    'case-lesional skin': 'skin-ADL',
    'case-nonlesional skin': 'skin-ADNL', 
    'control-nonlesional skin': 'skin-H',
    'case-anterior nares': 'nares-AD',
    'control-anterior nares': 'nares-H'
})

# Add AD status column based on group values
metadata['ad_status'] = metadata['group'].apply(lambda x: 'AD' if x.split('-')[-1].startswith('AD') else 'H')

metadata

,PlateNumber,PlateLocation,i5,i5Sequence,i7,i7Sequence,identifier,Sequence,Plate ID,Well location,...,enrolment_season,hiv_exposure,hiv_status,household_size,o_scorad,FWD_filepath,REV_filepath,group,microbiome_type,ad_status
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1,A1,SA501,ATCGTACG,SA701,CGAGAGTT,SA701SA501,CGAGAGTT-ATCGTACG,1.010000e+21,A1,...,Autumn,Unexposed,negative,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin,AD
900221,1,B1,SA502,ACTATCTG,SA701,CGAGAGTT,SA701SA502,CGAGAGTT-ACTATCTG,1.010000e+21,B1,...,Winter,Unexposed,negative,7.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin,AD
Ca010EBL,1,C1,SA503,TAGCGAGT,SA701,CGAGAGTT,SA701SA503,CGAGAGTT-TAGCGAGT,1.010000e+21,C1,...,Spring,Unexposed,negative,7.0,21,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin,AD
900460,1,D1,SA504,CTGCGTGT,SA701,CGAGAGTT,SA701SA504,CGAGAGTT-CTGCGTGT,1.010000e+21,D1,...,Spring,Unexposed,NaN,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin,AD
900051,1,E1,SA505,TCATCGAG,SA701,CGAGAGTT,SA701SA505,CGAGAGTT-TCATCGAG,1.010000e+21,E1,...,Autumn,Unexposed,negative,7.0,41,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin,AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca006ONL2,6,H1,SA508,GACACCGT,SB701,CTCGACTT,SB701SA508,CTCGACTT-GACACCGT,1.010000e+21,H1,...,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin,AD
Ca006ONNL,6,F2,SA506,CGTGAGTG,SB702,CGAAGTAT,SB702SA506,CGAAGTAT-CGTGAGTG,1.010000e+21,F2,...,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL,skin,AD
Ca006ONNL2,6,H2,SA508,GACACCGT,SB702,CGAAGTAT,SB702SA508,CGAAGTAT-GACACCGT,1.010000e+21,H2,...,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL,skin,AD


In [12]:
# Read in table at collapsed genera level
taxa_level = 'Genus'
biom_path = f'../Data/Tables/Absolute_Abundance_Tables/209766_filtered_by_prevalence_1pct_rare_{taxa_level}-ASV-non-collapse.biom'

biom_tbl = load_table(biom_path)
df = pd.DataFrame(biom_tbl.to_dataframe().T)

# delete the prefix from the 
df.index = df.index.str.replace('15564.', '')

# Map pid, group, and microbiome_type from metadata to df based on matching indexes
df['pid'] = metadata.loc[df.index, 'pid']
df['group'] = metadata.loc[df.index, 'group']
df['microbiome_type'] = metadata.loc[df.index, 'microbiome_type']
df

,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-3,g___ASV-3,g__Cutibacterium_ASV-1,g___ASV-16,g___ASV-25,g___ASV-27,g__Cutibacterium_ASV-2,...,g___ASV-126,g__Leptotrichia_A_993758_ASV-16,g__Capnocytophaga_820688_ASV-7,g__UBA6175_ASV-2,g__Comamonas_F_589250_ASV-4,g__Bosea_ASV-2,g___ASV-119,pid,group,microbiome_type
900344,156.0,95.0,23.0,17.0,1.0,2.0,1.0,0,0,0,...,0,0,0,0,0,0,0,Co-130-MM,skin-H,skin
900459,21.0,30.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Ca-145-LM,nares-AD,nares
900221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Ca-101-ID,skin-ADL,skin
900570,18.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Ca-046-MT,skin-ADNL,skin
900092,174.0,104.0,10.0,1.0,0,0,0,0,2.0,0,...,0,0,0,0,0,0,0,Ca-023-EJ,nares-AD,nares
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003972,38.0,21.0,0,0,1.0,18.0,0,0,0,12.0,...,0,0,0,0,0,0,0,Ca-125-IM,skin-ADL,skin
900097,3.0,0,0,0,0,3.0,0,0,0,0,...,0,0,0,0,0,0,0,Ca-025-AC,skin-ADNL,skin
900498,4.0,6.0,0,0,0,13.0,0,6.0,0,13.0,...,0,0,0,0,0,0,0,Ca-157-LM,skin-ADNL,skin
900276,0,0,10.0,0,0,69.0,0,0,0,26.0,...,0,0,0,0,0,0,0,Ca-111-IN,skin-ADL,skin


In [13]:
print(f'Number of unique individuals: ' + str(df['pid'].nunique()))

Number of unique individuals: 197


In [14]:
# --- STEP 1: Prepare ---
# Only keep numeric columns (i.e., ASV features)
taxa_cols = df.select_dtypes(include=[np.number]).columns.tolist()


# Split into skin and nares samples
skin_samples = df[df['microbiome_type'] == 'skin']
nares_samples = df[df['microbiome_type'] == 'nares']

# --- STEP 2: Separate by disease group ---

# AD individuals
skin_ADL_samples = skin_samples[skin_samples['group'] == 'skin-ADL']
nares_AD_samples = nares_samples[nares_samples['group'] == 'nares-AD']

# Healthy individuals
skin_H_samples = skin_samples[skin_samples['group'] == 'skin-H']
nares_H_samples = nares_samples[nares_samples['group'] == 'nares-H']

# --- STEP 3: Find taxa for each group ---

# For AD
skin_present_AD = skin_ADL_samples[taxa_cols].gt(0).any()
nares_present_AD = nares_AD_samples[taxa_cols].gt(0).any()

skin_taxa_AD = set(skin_present_AD[skin_present_AD].index)
nares_taxa_AD = set(nares_present_AD[nares_present_AD].index)

# For Healthy
skin_present_H = skin_H_samples[taxa_cols].gt(0).any()
nares_present_H = nares_H_samples[taxa_cols].gt(0).any()

skin_taxa_H = set(skin_present_H[skin_present_H].index)
nares_taxa_H = set(nares_present_H[nares_present_H].index)

In [15]:
# Get unique patient IDs for each group
skin_ADL_pids = set(skin_ADL_samples['pid'].unique())
nares_AD_pids = set(nares_AD_samples['pid'].unique())

# Count unique patients
n_AD = len(skin_ADL_pids.union(nares_AD_pids))

In [16]:
# Get unique patient IDs for each group
skin_H_pids = set(skin_H_samples['pid'].unique())
nares_H_pids = set(nares_H_samples['pid'].unique())

# Count unique patients
n_H = len(skin_H_pids.union(nares_H_pids))

In [ ]:
# --- STEP 4: Plot Venn diagrams side by side ---
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns

# Colors
skin_color = '#2a00ff'
nares_color = '#ffa501'
overlap_color = '#955280'

# --- Venn for AD Individuals ---
plt.sca(axes[0])  # set current axis
v_AD = venn2(
    [skin_taxa_AD, nares_taxa_AD],
    set_labels=('', ''),
    set_colors=(skin_color, nares_color),
    alpha=0.5
)

# Customize font sizes and outlines
for label in v_AD.subset_labels:
    if label:
        label.set_fontsize(16)

region_colors = {
    '10': skin_color,
    '01': nares_color,
    '11': overlap_color
}
for region_id, color in region_colors.items():
    patch = v_AD.get_patch_by_id(region_id)
    if patch:
        patch.set_linewidth(2)
        patch.set_edgecolor(color)

axes[0].set_title(f'AD Individuals (n=107)', fontsize=18, y=0.94)

# --- Venn for Healthy Individuals ---
plt.sca(axes[1])  # set current axis
v_H = venn2(
    [skin_taxa_H, nares_taxa_H],
    set_labels=('', ''),
    set_colors=(skin_color, nares_color),
    alpha=0.5
)

for label in v_H.subset_labels:
    if label:
        label.set_fontsize(16)

for region_id, color in region_colors.items():
    patch = v_H.get_patch_by_id(region_id)
    if patch:
        patch.set_linewidth(2)
        patch.set_edgecolor(color)

axes[1].set_title(f'Healthy Individuals (n=88)', fontsize=18, y=0.94)

# --- Add Legend ---
handles = [
    mpatches.Patch(color=skin_color, alpha=0.5, label='Skin only'),
    mpatches.Patch(color=overlap_color, alpha=0.5, label='Both'),
    mpatches.Patch(color=nares_color, alpha=0.5, label='Nares only')
]

fig.legend(
    handles=handles,
    loc='lower center',
    bbox_to_anchor=(0.5, 0.02),
    ncol=3,
    frameon=False,
    fontsize=16
)

# --- Final layout tweaks ---
plt.tight_layout()
plt.subplots_adjust(top=0.88, bottom=0.15)

# --- Supertitle ---
fig.suptitle('ASV Overlap Between Skin and Nares', fontsize=20, y=0.99)

# --- Annotation data ---
# You already computed these
shared_AD = len(skin_taxa_AD & nares_taxa_AD)
total_AD = len(skin_taxa_AD | nares_taxa_AD)

shared_H = len(skin_taxa_H & nares_taxa_H)
total_H = len(skin_taxa_H | nares_taxa_H)

# Run Fisher's test again if needed
from scipy.stats import fisher_exact
table = [[shared_AD, total_AD - shared_AD], [shared_H, total_H - shared_H]]
oddsratio, p_value = fisher_exact(table)

# Add p-value and odds ratio centrally below both
fig.text(
    0.5, 0.13,
    f"Fisher's Exact Test: p < 0.0001, Odds Ratio = {oddsratio:.2f}",
    ha='center', fontsize=14
)

# plt.savefig('../Figures/Main/Fig_3A.png', dpi=600)
plt.savefig('../Figures/Main/JPG/Fig_3A.jpg', dpi=600)


In [18]:
# --- For AD group ---
shared_AD = len(skin_taxa_AD & nares_taxa_AD)
total_AD = len(skin_taxa_AD | nares_taxa_AD)

# --- For Healthy group ---
shared_H = len(skin_taxa_H & nares_taxa_H)
total_H = len(skin_taxa_H | nares_taxa_H)

# --- Build 2x2 contingency table ---
table = [
    [shared_AD, total_AD - shared_AD],
    [shared_H, total_H - shared_H]
]

# --- Fisher's Exact Test ---
oddsratio, p_value = fisher_exact(table)

# --- Output Results ---
print(f"Fisher's Exact Test results:")
print(f"Contingency Table: {table}")
print(f"Odds Ratio: {oddsratio:.2f}")
print(f"p-value: {p_value:.8f}")

Fisher's Exact Test results:
Contingency Table: [[369, 299], [254, 383]]
Odds Ratio: 1.86
p-value: 0.00000003
